In [ ]:
import os

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import torch
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

#from decoders import (StackedUnidirectionalLSTMDecoder, StackedUnidirLSTMDenseDecoder, 
#                      StackedUnidirectionalLSTMDecoderwithEmbedding)
#from encoders import StackedUnidirectionalLSTMEncoder, StackedBidirectionalLSTMEncoder
#from models import AutoEncoder
from dataloaders import load_data, create_loaders
from training import main

In [ ]:
import models
import encoders
import decoders

In [ ]:
dataset_path = '/projects/bassline_transcription/data/datasets'
M = 8 

data_params = {'dataset_path': dataset_path,
               'scale_type': 'min',
               'M': M,
               'batch_size': 8}

X = load_data(data_params)

K = X.max()+1 # ???????
sequence_length = X.shape[1]

encoder_params = {'num_embeddings': K,
                  'embedding_dim': 16, #??
                  'hidden_size': 128,
                  'dropout': 0,  
                  'num_layers': 4,              
                  'batch_size': data_params['batch_size'],
                  'device':device}

decoder_params = {'input_size': encoder_params['hidden_size'],
                  'output_size': K,
                 'num_layers': encoder_params['num_layers'], 
                 'dropout': 0,
                 'batch_size': data_params['batch_size'],
                 'sequence_length': sequence_length,
                 'device':device}

train_params = {'batch_size': data_params['batch_size'],
                'N_epochs': 10}

all_params = {'encoder_params': encoder_params, 'decoder_params':decoder_params, 'train_params':train_params}

In [ ]:
encoder_params = {'num_embeddings': K,
                  'embedding_dim': 16, 
                  'hidden_size': 128,
                  'dropout': 0.2,  
                  'num_layers': 2,              
                  'batch_size': data_params['batch_size'],
                  'device':device}

decoder_params = {'num_embeddings': K,
                  'embedding_dim': 16,
                  'hidden_size': 64,                  
                  'num_layers': encoder_params['num_layers'], 
                  'dropout': 0.2,
                  'batch_size': data_params['batch_size'],
                  'sequence_length': sequence_length,
                  'device':device}

train_params = {'batch_size': data_params['batch_size'],
                'N_epochs': 10}

all_params = {'encoder_params': encoder_params, 'decoder_params':decoder_params, 'train_params':train_params}

model_path = os.path.join('models', model_name+'.pt')

torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_loss': train_loss,
            'test_loss': test_loss
            }, model_path)

In [ ]:
#encoder = StackedUnidirectionalLSTMEncoder(**encoder_params)
encoder = encoders.StackedBidirectionalLSTMEncoder(**encoder_params).to(device)

#StackedUnidirectionalLSTMDecoder(**decoder_params)
#decoder = StackedUnidirLSTMDenseDecoder(**decoder_params)
decoder = decoders.StackedUnidirectionalLSTMDecoderwithEmbedding(**decoder_params).to(device)

#model = models.AutoEncoder(encoder, decoder).to(device)
model = models.AE(encoder, decoder, device).to(device)

print(model)
print('Number of parameters: {}'.format(sum([parameter.numel() for parameter in model.parameters()])))

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

train_loader, test_loader = create_loaders(X, data_params)

train_losses, test_losses = main(model, train_loader, test_loader, optimizer, criterion, train_params, device)

In [ ]:
import datetime as dt
from training import train, test
from tqdm import tqdm
import wandb
#wandb.login()

In [ ]:
test_loss = test(model, test_loader, criterion, device)

for epoch in tqdm(range(train_params['N_epochs'])):

    train_loss = train(model, train_loader, optimizer, criterion, device)
    
    if epoch+1 % 5:
        print('Epoch: {}, train_loss:{:.6f}'.format(epoch+1, train_loss))

test_loss = test(model, test_loader, criterion, device)
print('Test Loss After Training: {:.6f}'.format(test_loss))

In [ ]:
project_name='NeuralBasslineGenerator'

model_name = dt.datetime.strftime(dt.datetime.now(),"%H_%M__%d_%m_%Y")

with wandb.init(project=project_name, name=model_name, config=all_params):

    test_loss = test(model, test_loader, criterion, device)
    wandb.log({'test_loss': test_loss})

    for epoch in tqdm(range(train_params['N_epochs'])):

        train_loss = train(model, train_loader, optimizer, criterion, device)
        wandb.log({'train_loss': train_loss})

        if epoch+1 % 5:
            print('Epoch: {}, train_loss:{:.6f}'.format(epoch+1, train_loss))

    test_loss = test(model, test_loader, criterion, device)
    print('Test Loss After Training: {:.6f}'.format(test_loss))
    wandb.log({'test_loss': test_loss})